In [1]:

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import distutils

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'C:/Users/MikeEd/Deep-Learning-Push-Up-Counter-master/data/train',
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=16,
    class_mode='categorical',
    shuffle=True,
    seed=42)

validation_generator = test_datagen.flow_from_directory(
    'C:/Users/MikeEd/Deep-Learning-Push-Up-Counter-master/data/validation',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical')

C:\Users\MikeEd\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Found 376 images belonging to 3 classes.
Found 84 images belonging to 3 classes.


In [2]:
def create_model():
  model = tf.keras.models.Sequential()
  #model.add(tf.keras.Input(shape=(64,64,3)))
  #model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(6, (3, 3), padding='same', activation='relu',input_shape=(64,64,3)))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  
  model.add(tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

  #model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

  #model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))  

  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(512))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(128))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(64))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(3))
  model.add(tf.keras.layers.Activation('softmax'))
  return model

In [21]:
model = create_model()
model.compile(
      optimizer=tf.keras.optimizers.Adam(lr=1e-3),
      loss='categorical_crossentropy',
      metrics=['categorical_accuracy'])

model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    validation_freq=1
)

model.save('model.h5', overwrite=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
24/24 [==============================] - 38s 2s/step - loss: 1.0125 - categorical_accuracy: 0.4574 - val_loss: 0.6824 - val_categorical_accuracy: 0.7619
Epoch 2/10
24/24 [==============================] - 20s 816ms/step - loss: 0.6197 - categorical_accuracy: 0.7686 - val_loss: 0.3403 - val_categorical_accuracy: 0.8333
Epoch 3/10
24/24 [==============================] - 22s 920ms/step - loss: 0.4693 - categorical_accuracy: 0.8537 - val_loss: 0.2910 - val_categorical_accuracy: 0.8929
Epoch 4/10
24/24 [==============================] - 20s 829ms/step - loss: 0.4058 - categorical_accuracy: 0.8617 - val_loss: 0.3716 - val_categorical_accuracy: 0.8690
Epoch 5/10
24/24 [==============================] - 17s 708ms/step - loss: 0.4110 - categorical_accuracy: 0.8511 - val_loss: 0.2548 - val_categorical_accuracy: 0.9405
Epoch 6/10
24/24 [==============================] - 18s 763ms/step - loss: 0.3746 - categorical_accuracy: 0.8883 - val_l

In [6]:
model = tf.keras.models.load_model('model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [21]:
import cv2
import numpy as np
#cap = cv2.VideoCapture("F:/!_MTECH/3rd sem/Deep-Learning-Push-Up-Counter-master/test_video.mov")
cap = cv2.VideoCapture("F:/!_MTECH/3rd sem/Deep-Learning-Push-Up-Counter-master/test2.mp4")
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
out = cv2.VideoWriter('output_count.avi',fourcc, 20.0,(int(cap.get(3)),int(cap.get(4))))

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
i= 0
prediction_str = ""
repetitions = 0
up = 0
down = 0
no_move = 0
current_move = 0
initial = -1
while(cap.isOpened()):
    i+=1
    
    ret, frame2 = cap.read()
    if not(ret): break
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    image = cv2.resize(rgb, (64, 64))
    image = image.reshape((1,) + image.shape)
    image = image/255.0
    prediction = np.argmax(model.predict(image), axis=-1)[0]
    
    if prediction == 0:
        down +=1 
        if down == 3:
          if initial == -1:
            initial = 0
          if current_move == 2:
            repetitions+=1
          current_move = 0
        elif down > 0:
          up = 0
          no_move = 0
    elif prediction == 2:
        up += 1
        if up == 3 and initial != -1:
          current_move = 2
        elif up > 1:
          down = 0 
          no_move = 0
    else:
        no_move += 1
        if no_move == 15:
          current_move = 1
        elif no_move > 10:
          up = 0
          down = 0 
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,400)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 5
    cv2.putText(frame2, "Repetitions: "+ str(repetitions),bottomLeftCornerOfText,font, fontScale,fontColor,lineType)
    out.write(frame2)
    prvs = next

print("Video Generated")
out.release()
cap.release()
cv2.destroyAllWindows()

Video Generated
